In [3]:
import os
import torch
import numpy as np
from matplotlib import pyplot as plt

In [12]:
from torch.utils.data import Dataset
import torchvision.transforms as T
import PIL.Image as Image
import numpy as np
import torch

class CustomDataset(Dataset):
    def __init__(self, img_list_file='/mnt/data/ILSVRC/Data/train_img_paths.txt', transforms=None, std=None, crop_size=224, device='cpu') -> None:
        super().__init__()
        with open(img_list_file, 'r') as f:
            self.img_paths = f.readlines()
        self.crop_size = crop_size
        self.transform = transforms or  T.Compose([
                T.ToTensor(),
                T.CenterCrop(size=224)
            ])
        self.std = std
        self.device = device
    
    def __add_noise__(self, img):
        std = self.std or np.random.uniform(0, 1)
        img = np.array(img)

        noise = np.random.normal(0, std, img.shape)
        noisy_img = np.clip(img + noise, 0, 255)
        return noisy_img
    
    def __get_img__(self, idx):
        img_path = self.img_paths[idx].strip()
        img = Image.open(img_path).convert("L")
        noisy_img = self.__add_noise__(img)

        img = self.transform(img)
        noisy_img = self.transform(noisy_img)

        return img, noisy_img
    
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img_1, noisy_1 = self.__get_img__(idx)

        # get a random image
        target_idx = np.random.randint(0, len(self.img_paths))
        img_2, noisy_2 = self.__get_img__(target_idx)

        noisy = torch.cat((noisy_1, noisy_2)).to(torch.float)
        clean = torch.cat((img_1, img_2), dim=0).to(torch.float)
        return noisy.to(device=self.device), clean.to(device=self.device)

imagenet_path = '/mnt/data/ILSVRC/Data'
train_path = os.path.join(imagenet_path, "train")
image_list = os.path.join(imagenet_path, "train_img_paths.txt")
dataset = CustomDataset(image_list, device='cuda')
data = next(iter(dataset))


In [ ]:
import torch
from torch.optim import Adam
from torch.nn import MSELoss
from torch.utils.data import DataLoader
from tqdm import tqdm

# from dataset import CustomDataset
from smaller_model import UNet

device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32
dataset = CustomDataset(device=device)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


model = UNet().to(device=device)

lr = 1e-3
optim = Adam(model.parameters(), lr=lr)
loss_fn = MSELoss()
log_every = 10

n_epochs = 1

for epoch in range(n_epochs):
    for noisy, clean in tqdm(dataloader):
        optim.zero_grad()
        pred = model(noisy.to(torch.float))

        loss = loss_fn(pred, clean)
        loss.backward()
        optim.step()

        if epoch > 0 and epoch % log_every == 0:
            print(f'Loss: {loss.item()}')

In [1]:
data_path = "/fs/vulcan-datasets/imagenet/train"

import os
import glob

files_ = []
for root, dir, files in os.walk(data_path):
    for file in files:
        if file.lower().endswith(".jpeg"):
            files_.append(os.path.append(root, file))

with open('./vulcan_files.txt', 'w') as f:
    f.writelines(files_)

In [21]:
with open('./vulcan_files.txt', 'r') as f:
    img_paths = f.readlines()

img_paths = [img_path.strip() for img_path in img_paths]
corrected = []

import re

pattern = r"/train/(n\d+)_(\.*)"
for name in img_paths:
    new_path = re.sub(pattern, r"/train/\1/\1_\2", name)
    corrected.append(new_path)

with open('./vulcan_files_2.txt', 'w') as f:
    f.write("\n".join(corrected))

In [23]:
for img_path in corrected:
    img_path = img_path.strip()
    image = Image.open(img_path)
    image.close()


KeyboardInterrupt: 

In [11]:
original_string = "/fs/vulcan-datasets/imagenet/train/n01440764_10026.JPEG"

# Define a regular expression pattern to capture the desired value
pattern = r"/train/(\w+)_(\.*)"

# Replace the pattern with the modified string
modified_string = re.sub(pattern, r"/train/\1/\1_\2", original_string)

print(modified_string)


/fs/vulcan-datasets/imagenet/train/n01440764/n01440764_10026.JPEG
